<a href="https://colab.research.google.com/github/yeonsikC/Deep-Learning-with-Keras-Founders/blob/master/Deep_Learning_with_Keras_Founders_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 머신러닝

## 4. 1. 여러 검증 방법들

In [0]:
#홀드아웃
num_validation_samples = 10000

np.random.shuffle(data)

validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

trainin_data = data[:]

model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

# 여기에서 모델을 튜닝하고,
# 다시 훈련하고, 평가하고, 또 다시 튜닝하고 ...

# 하이퍼파라미터 튜닝이 끝나면 테스트 데이터를 제외한 모든 데이터를 통해 모델을 다시 훈련시킨다.
model = get_model()
model.train(np.concateenate([trainin_data,
                             validation_data]))
test_score = model.evaluate(test_data)

In [0]:
#K-Fold
k = 4
num_validation_samples = len(data) // k
 
np.random.shuffle(data)

validation_scores = []
for fold in range(k):
  validation_data = data[num_validation_samples * fold:
                         num_validation_samples * (fold + 1)]
  training_data = data[:num_validation_samples * fold] +
                  data[num_validation_samples * (fold + 1):]

  model = get_model()
  model.train(training_data)
  validation_score = model.evaluate(validation_data)
  validation_scores.append(validation_score)

  validation_score = np.average(validation_scores) # 검증 점수 : k개 폴드의 검증 점수 평균

  model = get_model()
  model.train(data)
  test_score = model.evaluate(test_data)

  #해당 검증은 사이킷런의 cross_validate() 함수를 이용하면 된다.
  #다만, 케라스와 호환되도록 KerasClassifier나 KerasRegressor 클래스로 모델을 감싸야 합니다.

##4. 2. 데이터 전처리

- 벡터화 : 신경망에서 모든 입력과 타깃은 부동 소수 데이터로 이루어진 텐서여야 한다.
- 정규화 : 작은 값을 취해야 함(대부분 0~1), 균일해야 한다.
  * 각 특성별로 N(0,1)을 따라야 좋다. -> x-=x.mean(axis=0); x/= x.std(axis=0)
- 특성공학 : ex) 시계바늘 등 미리 하드코딩 된 정보를 모델에 주입시키면 적은 데이터로 더 좋은 효율을 낼 수 있다.

##4. 3. 과대적합과 과소적합 / 네트워크 크기 축소



In [0]:
#영화 리뷰 분류 모델, 하이퍼파라미터 및 네트워크 조절
from keras import models
from keras import layers

#원본모델
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
#작은 용량의 모델
model = models.Sequential()
model.add(layers.Dense(6, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
#큰 용량의 모델
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
#가중치 규제 L1 & L2
from keras import regularizers
model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#l2(0.001)은 가중치 행렬의 모든 원소를 제곱하고 0.001을 곱하여 네트워크의 전체 손실에 더해 진다는 의미이다.
#이 페널티 항은 훈련할 때만 추가된다. (과대적합에 더 견딘다.)

In [0]:
#가중치 규제 선택 가능
from keras import regularizers

regularizers.l1(0.001) # L1 규제
regularizers.l1_l2(l1=0.001, l2=0.001) #L1과 L2 규제 병행

In [0]:
#Dropout
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


- 신경망에서 과대적합을 방지하기 위한 방법
  * 훈련 데이터를 더 늘린다.
  * 네트워크의 용량을 감소시킨다. (은닉층, 유닛 수 등)
  * 가중치 규제를 추가한다.
  * 드롭아웃을 추가한다.